In [24]:
import numpy as np
import matplotlib.pyplot as plt
import src.mogli as mogli
import os
from ipywidgets import IntProgress
from IPython.display import display, clear_output
from PIL import Image

from src.CameraPoseVisualizer import CameraPoseVisualizer
# import numpy as np

import importlib
import json
import time


In [25]:
bandgaps = np.loadtxt('Datasets/bandgaps.csv')
cods = np.loadtxt('Datasets/CODids.csv', dtype=int)
materials = mogli.read('Datasets/structures.xyz')

In [ ]:
def circle(radius, n):
    theta = np.linspace(0, 2*np.pi, n)
    x = radius*np.cos(theta)
    y = radius*np.sin(theta)
    z = np.zeros_like(theta)
    # put x,y, and z as columns in a matrix

    return np.column_stack((x.flatten(), y.flatten(), z.flatten()))
# x = circle(100, 20)

# plt.figure(figsize=(10,10))
# plt.plot(x[:,0], x[:,1], 'o')


In [ ]:
def sphere (radius, n):
    theta = np.linspace(0, 2*np.pi, n)
    phi = np.linspace(0, np.pi, n)
    x = radius*np.outer(np.cos(theta), np.sin(phi)).flatten()
    y = radius*np.outer(np.sin(theta), np.sin(phi)).flatten()
    z = radius*np.outer(np.ones(np.size(theta)), np.cos(phi)).flatten()
    
    
    mask = np.logical_and(x != 0, y != 0)
    x = x[mask]
    y = y[mask]
    z = z[mask]
    
    # print(x.shape)
    
    return np.column_stack((x,y,z))
# s = sphere(25, 10)
# # plot points
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(s[:,0], s[:,1], s[:,2], c='b', marker='o')

In [ ]:
class Cameraframe():
    def __init__(self,filePath,transform_matrix, sharpness=5725):
        self.file_path = filePath
        self.sharpness= sharpness
        self.transform_matrix = transform_matrix
    def toJson(self):
        return {
            "file_path": self.file_path,
            # "sharpness": self.sharpness,
            "transform_matrix": self.transform_matrix.tolist()
        }
    
class dataset():
    def __init__(self, w, h ,angle_degrees = 45):        
        self.camera_angle_x= np.radians(angle_degrees)
        self.camera_angle_y= np.radians(angle_degrees)   
        self.w= w
        self.h= h
        self.frames = []
    def toJson(self):
        return {
            "camera_angle_x": self.camera_angle_x,
            "camera_angle_y": self.camera_angle_y,           
            "cx": self.w / 2,
            "cy": self.h / 2,
            "w": self.w,
            "h": self.h,
            "frames": [frame.toJson() for frame in self.frames]             
        }
    def save(self, path):
        with open(path, 'w') as outfile:
            json.dump(self.toJson(), outfile)
        

In [ ]:
def run(scene_path, train_steps):
    snapshot_path = os.path.join(scene_path, f"snapshot.ingp")    
    !python ./instant-ngp/scripts/run.py {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path}  --save_raw_volumes
def open_snapshot(scene_path):
    train_steps = 10
    snapshot_path = os.path.join(scene_path, f"snapshot.ingp")
    !python ./instant-ngp/scripts/run.py {scene_path} --n_steps {train_steps} --load_snapshot {snapshot_path} --gui

def save_raw_volumes(scene_path):
    snapshot_path = os.path.join(scene_path, f"snapshot.ingp")    
    !python ./instant-ngp/scripts/run.py {scene_path} --load_snapshot {snapshot_path} --save_raw_volumes

In [ ]:
def viewCameras(poses ):
    visualizer = CameraPoseVisualizer([-10, 10], [-10, 10], [-10, 10])
    for pose in poses:
        visualizer.extrinsic2pyramid(pose.transform_matrix, 'c', 10)
    visualizer.show()   
    

In [ ]:
bonds_param = 1.5
width = 100
height = 100
distance = 25
formatFile = "jpg"
train_steps = 4000  
start_molecue = 0 
path = f'E:\Datasets\BandGap\{width}x{height}_BP{bonds_param}/'
if not os.path.exists(path):
    os.makedirs(path)    
for i, molecule in enumerate(materials[start_molecue:]):  
    # if i == 34:
    #     break
    start_time = time.time()
    i+=start_molecue
    print(f'molecule {i} stated')  
    pathmolecule = f'{path}/{i}'
    if not os.path.exists(pathmolecule):
        os.makedirs(pathmolecule)   
    Nerf = dataset(width,height)
    data  = {}
    sp = sphere(distance, 15)
    for n , (x,y,z) in enumerate(sp):              
        img_path = f"{pathmolecule}/molecula_x_{i}_{n}.{formatFile}"
        camerapos = np.array([x,y,z])
        camera = mogli.export(molecule,img_path,bonds_param=bonds_param,camera=((x,y,z),(0, 0, 0),(0 , 0 , 1)), width=width, height=height,distance=distance,scale=2)
        # camera[:3,3] =  camerapos / distance
        Nerf.frames.append(Cameraframe(f"molecula_x_{i}_{n}.{formatFile}",camera))
    Nerf.save(f"{pathmolecule}/nerf.json")   
    # viewCameras(Nerf.frames)    
     
    # open_snapshot(pathmolecule) 
    # save_raw_volumes(pathmolecule)
    
    run(pathmolecule, train_steps)
    clear_output(wait=True)  

    print(f'molecule {i} finished--- {time.time() - start_time}seconds ---'  )
    break

In [ ]:

img_path = r"E:\Datasets\BandGap\100x100_BP1.5\1\volume_raw\128x128x128_0.bin"

data =None
with open(img_path, "rb") as f:
    data = f.read()

data = np.frombuffer(data, dtype=np.float32)
size = np.cbrt(data.shape[0]/4).astype(int)
data = data.reshape((size,size,size,4))
data.shape
for i in range(size):
    img = data[i]
    plt.imshow(img)
    
    plt.show()
